# RHMetaD Regression Tutorial

This notebook demonstrates the RHMetaD regression model on simulated data.
All code runs locally and does not require external datasets.

Quick run recipe:
```bash
conda create -n metadpy-demo python=3.11 -y
conda activate metadpy-demo
pip install "git+https://github.com/heyifei1984/metadpy.git@main"
pip install jupyter matplotlib arviz ipykernel
python -m ipykernel install --user --name metadpy-demo --display-name "Python (metadpy-demo)"
jupyter notebook
```
In Jupyter, select the `Python (metadpy-demo)` kernel.


## Setup

Please install from the main branch via pip (no git clone or editable installs):
```bash
pip install "git+https://github.com/heyifei1984/metadpy.git@main"
```


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import arviz as az
from IPython.display import display

import metadpy
from metadpy.bayesian import rhmetad


In [ ]:
print('metadpy version:', metadpy.__version__)

## Simulation A: single condition

We simulate S=20 subjects, K=4 ratings, and 100 trials per subject.
We generate covariate values `x_s ~ Normal(0, 1)` and simulate
`log(Mratio)_s = M0 + beta * x_s + noise`.

Counts are generated from the RHMetaD likelihood (CR/FA/M/H multinomials).


In [ ]:
from scipy.stats import norm

def _clamp_probs(p, tol=1e-5):
    p = np.clip(p, tol, None)
    return p / p.sum()


def simulate_counts(rng, n_ratings, x, beta, mu_logmratio, d1, c1, c_s1, c_s2, n_cr, n_fa, n_m, n_h):
    n_subj = x.shape[0]
    nR_S1 = np.zeros((n_subj, 2 * n_ratings), dtype=int)
    nR_S2 = np.zeros((n_subj, 2 * n_ratings), dtype=int)

    for idx in range(n_subj):
        log_mratio = mu_logmratio + x[idx, 0] * beta
        meta_d = np.exp(log_mratio) * d1
        s1_mu = -meta_d / 2.0
        s2_mu = meta_d / 2.0

        phi_c1_s1 = norm.cdf(c1 - s1_mu)
        phi_c1_s2 = norm.cdf(c1 - s2_mu)
        phi_cs1_s1 = norm.cdf(c_s1 - s1_mu)
        phi_cs1_s2 = norm.cdf(c_s1 - s2_mu)
        phi_cs2_s1 = norm.cdf(c_s2 - s1_mu)
        phi_cs2_s2 = norm.cdf(c_s2 - s2_mu)

        c_area_rs1 = phi_c1_s1
        i_area_rs2 = 1.0 - phi_c1_s1
        i_area_rs1 = phi_c1_s2
        c_area_rs2 = 1.0 - phi_c1_s2

        p_cr = np.concatenate((
            phi_cs1_s1[:1] / c_area_rs1,
            (phi_cs1_s1[1:] - phi_cs1_s1[:-1]) / c_area_rs1,
            np.array([(phi_c1_s1 - phi_cs1_s1[-1]) / c_area_rs1]),
        ))
        p_fa = np.concatenate((
            np.array([(phi_cs2_s1[0] - phi_c1_s1) / i_area_rs2]),
            (phi_cs2_s1[1:] - phi_cs2_s1[:-1]) / i_area_rs2,
            np.array([(1.0 - phi_cs2_s1[-1]) / i_area_rs2]),
        ))
        p_m = np.concatenate((
            phi_cs1_s2[:1] / i_area_rs1,
            (phi_cs1_s2[1:] - phi_cs1_s2[:-1]) / i_area_rs1,
            np.array([(phi_c1_s2 - phi_cs1_s2[-1]) / i_area_rs1]),
        ))
        p_h = np.concatenate((
            np.array([(phi_cs2_s2[0] - phi_c1_s2) / c_area_rs2]),
            (phi_cs2_s2[1:] - phi_cs2_s2[:-1]) / c_area_rs2,
            np.array([(1.0 - phi_cs2_s2[-1]) / c_area_rs2]),
        ))

        p_cr = _clamp_probs(p_cr)
        p_fa = _clamp_probs(p_fa)
        p_m = _clamp_probs(p_m)
        p_h = _clamp_probs(p_h)

        cr_counts = rng.multinomial(n_cr, p_cr)
        fa_counts = rng.multinomial(n_fa, p_fa)
        m_counts = rng.multinomial(n_m, p_m)
        h_counts = rng.multinomial(n_h, p_h)

        nR_S1[idx, :n_ratings] = cr_counts
        nR_S1[idx, n_ratings:] = fa_counts
        nR_S2[idx, :n_ratings] = m_counts
        nR_S2[idx, n_ratings:] = h_counts

    return nR_S1, nR_S2


In [ ]:
rng = np.random.default_rng(123)
n_subj = 20
n_ratings = 4
x = rng.normal(0.0, 1.0, size=(n_subj, 1))

beta = 0.6
mu_logmratio = -0.1
sigma_logmratio = 0.3
d1 = 1.5
c1 = 0.0
c_s1 = np.array([-1.5, -0.5, -0.1])
c_s2 = np.array([0.1, 0.5, 1.5])

noise = rng.standard_t(df=5, size=n_subj) * sigma_logmratio
log_mratio = mu_logmratio + beta * x[:, 0] + noise

nR_S1, nR_S2 = simulate_counts(
    rng=rng,
    n_ratings=n_ratings,
    x=x,
    beta=beta,
    mu_logmratio=mu_logmratio,
    d1=d1,
    c1=c1,
    c_s1=c_s1,
    c_s2=c_s2,
    n_cr=30,
    n_fa=20,
    n_m=20,
    n_h=30,
)

model_a, idata_a = rhmetad(
    nR_S1=nR_S1,
    nR_S2=nR_S2,
    nRatings=n_ratings,
    X=x,
    draws=300,
    tune=300,
    chains=2,
    random_seed=123,
    target_accept=0.9,
)


In [ ]:
logmratio_mean = idata_a.posterior['logMratio'].mean(dim=('chain', 'draw')).to_numpy()
beta_samples = idata_a.posterior['beta'].to_numpy().reshape(-1)

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].scatter(x[:, 0], logmratio_mean, c='k')
ax[0].set_xlabel('covariate x')
ax[0].set_ylabel('posterior mean log(Mratio)')
ax[0].set_title('Subjects')

az.plot_posterior({'beta': beta_samples}, hdi_prob=0.94, ax=ax[1])
ax[1].set_title('beta posterior (94% interval)')
plt.tight_layout()

beta_mean = beta_samples.mean()
beta_hdi = az.hdi(beta_samples, hdi_prob=0.94)
print('beta mean:', round(beta_mean, 3))
print('94% HDI:', [round(beta_hdi[0], 3), round(beta_hdi[1], 3)])
print('exp(beta) is the multiplicative effect on Mratio per +1 in x')


In [ ]:
summary_a = az.summary(
    idata_a,
    var_names=['mu_logMratio', 'beta', 'sigma_logMratio'],
    round_to=3,
)
display(summary_a[['mean', 'sd', 'hdi_3%', 'hdi_97%']])

print('ESS and R-hat (Simulation A):')
display(summary_a[['ess_bulk', 'ess_tail', 'r_hat']])


## Simulation B: two conditions

We simulate Condition A and B with different intercepts (M0).
Each condition is fit separately, then we compare beta posteriors and
the recovered log(Mratio) distributions.


In [ ]:
def fit_condition(mu_logmratio, beta, seed):
    rng = np.random.default_rng(seed)
    x = rng.normal(0.0, 1.0, size=(n_subj, 1))
    nR_S1, nR_S2 = simulate_counts(
        rng=rng,
        n_ratings=n_ratings,
        x=x,
        beta=beta,
        mu_logmratio=mu_logmratio,
        d1=d1,
        c1=c1,
        c_s1=c_s1,
        c_s2=c_s2,
        n_cr=30,
        n_fa=20,
        n_m=20,
        n_h=30,
    )
    model, idata = rhmetad(
        nR_S1=nR_S1,
        nR_S2=nR_S2,
        nRatings=n_ratings,
        X=x,
        draws=200,
        tune=200,
        chains=2,
        random_seed=seed,
        target_accept=0.9,
    )
    return x, idata

x_a, idata_b = fit_condition(mu_logmratio=-0.2, beta=0.6, seed=101)
x_b, idata_c = fit_condition(mu_logmratio=0.2, beta=0.6, seed=202)


In [ ]:
beta_a = idata_b.posterior['beta'].to_numpy().reshape(-1)
beta_b = idata_c.posterior['beta'].to_numpy().reshape(-1)

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].hist(beta_a, bins=30, alpha=0.6, label='Condition A')
ax[0].hist(beta_b, bins=30, alpha=0.6, label='Condition B')
ax[0].set_title('beta posterior comparison')
ax[0].legend()

logm_a = idata_b.posterior['logMratio'].mean(dim=('chain', 'draw')).to_numpy()
logm_b = idata_c.posterior['logMratio'].mean(dim=('chain', 'draw')).to_numpy()

ax[1].boxplot([logm_a, logm_b], labels=['A', 'B'])
ax[1].set_title('Recovered log(Mratio) by condition')
plt.tight_layout()


In [ ]:
summary_b = az.summary(
    idata_b,
    var_names=['mu_logMratio', 'beta', 'sigma_logMratio'],
    round_to=3,
)
summary_c = az.summary(
    idata_c,
    var_names=['mu_logMratio', 'beta', 'sigma_logMratio'],
    round_to=3,
)

print('Recovered parameters (Condition A):')
display(summary_b[['mean', 'sd', 'hdi_3%', 'hdi_97%']])
print('Recovered parameters (Condition B):')
display(summary_c[['mean', 'sd', 'hdi_3%', 'hdi_97%']])

print('ESS and R-hat (Condition A):')
display(summary_b[['ess_bulk', 'ess_tail', 'r_hat']])
print('ESS and R-hat (Condition B):')
display(summary_c[['ess_bulk', 'ess_tail', 'r_hat']])


## Notes & caveats

- Covariate shapes: (S, P) is preferred. MATLAB-style (P, S) is accepted and transposed.
  Square X is treated as (subjects x covariates) and is not transposed.
- Optional/experimental compile modes can be passed via `compile_mode` or `compile_kwargs`.
  Default remains CPU; no platform auto-detection.
